# MASTER - Notebook 2
### Matteo Grazioso 884055

In [177]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from datetime import datetime
import re
import folium
from folium.plugins import MarkerCluster
from pandas import Timestamp
import json
import warnings
warnings.filterwarnings('ignore')

import myfunctions as mf # Custom functions

In [178]:
# Disply all columns and all rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [179]:
# Find all txt files in the data folder
txt_files = mf.find_txt_files("data/processed")

# Choose a dataset from the list of txt files
selected_dataset = mf.choose_dataset(txt_files)

if selected_dataset:
    print(f"You selected the dataset {selected_dataset}")
else:
    print("No dataset selected.")

path  = selected_dataset

The following TXT files were found:
1. data/processed/dataset_cleaned_esportazioneCompleta.txt
2. data/processed/dataset_cleaned_esportazionePasqua23.txt
3. data/processed/dataset_cleaned_tempexport.txt
4. data/processed/dataset_cleaned_validazioni.txt
5. data/processed/old smaller/dataset_cleaned_esportazioneCompleta.txt
6. data/processed/old smaller/dataset_cleaned_validazioni.txt
You selected the dataset data/processed/dataset_cleaned_esportazioneCompleta.txt


In [180]:
# The file contains the data of the validation of tickets in the city of public transport of Venice.
# The file has been created by the Notebook 1.ipynb

df = pd.read_csv(path, header=0, sep='\t')

# Save the name of the file in a variable for future use extracting the name of the file from the path
file_name = path.split('_')[-1].split('.')[0]
subfolder = file_name

# Display the first 5 rows of the dataframe
df.head()

# Convert the column 'DATA' to datetime format
df['DATA'] = pd.to_datetime(df['DATA'], format='%Y-%m-%d')

In [181]:
# Open the dictionary of trajectories
def open_dict_trajectories(is_focus_on_ticket_code: bool = False, ticket_code: str = None, is_focus_on_tourists: bool = False, is_focus_on_workers: bool = False) -> dict:
    """
        This function opens the dictionary of trajectories eventually filtered by ticket code, tourists or workers.
        :param is_focus_on_ticket_code: if True, the dictionary will be filtered by ticket code
        :param ticket_code: the ticket code to filter the dictionary
        :param is_focus_on_tourists: if True, the dictionary will be filtered by tourists
        :param is_focus_on_workers: if True, the dictionary will be filtered by workers
        :return: the dictionary of trajectories eventually filtered
    """
    if is_focus_on_ticket_code:
        # Open the dictionary of trajectories filtered by ticket code
        with open('data/dictionaries/trajectories/' + subfolder + '/dict_trajectories_' + file_name + '_tc:' + ticket_code + '.json') as f:
            data = json.load(f)
        return data
    elif is_focus_on_tourists:
        # Open the dictionary of trajectories filtered by tourists
        with open('data/dictionaries/trajectories/' + subfolder + '/dict_trajectories_' + file_name + '_tourists.json') as f:
            data = json.load(f)
        return data
    elif is_focus_on_workers:
        # Open the dictionary of trajectories filtered by workers
        with open('data/dictionaries/trajectories/' + subfolder + '/dict_trajectories_' + file_name + '_workers.json') as f:
            data = json.load(f)
        return data
    else:
        # Open the dictionary of trajectories without filters
        with open('data/dictionaries/trajectories/' + subfolder + '/dict_trajectories_' + file_name + '.json') as f:
            data = json.load(f)
        return data

In [182]:
def extract_values_key(key: str) -> tuple:
    """
        This function extracts the values of the key.
        :param key: the key in the format (serial, day) where day can be None
        :return: the key in a tuple format (serial, day)
    """
    pattern = r"\((-?\d+),\s?(None|Timestamp\('.*'\))\)"
    match = re.match(pattern, key)
    if match:
        serial = int(match.group(1))
        # Timestamp is None if the second group is 'None', otherwise it is a Timestamp object not datetime
        timestamp = None if match.group(2) == 'None' else pd.Timestamp(match.group(2)[11:-2])
        return serial, timestamp
    else:
        # Print a messagge in red in a pretty format
        print('\033[91m' + 'The key {} is not in the correct format.'.format(key) + '\033[0m')
        return None, None


In [183]:
def get_rows_from_key(df_k: pd.DataFrame, key: tuple) -> pd.DataFrame:
    """
        This function returns the rows of the dataframe for the specified key.
        Note that the key is in the format (serial, day), where day can be None that means that the seriale doesn't change over the days.
        :param df_k: the dataframe
        :param key: the key
        :return: the row of the dataframe
    """
    # If the day is None, return the dataframe with the trajectories of the user
    if key[1] == None:
        print('The key is: {}'.format(key))
        return df_k[df_k['SERIALE'] == key[0]]
    # Otherwise, return the dataframe with the trajectories of the user in the specified day
    else:
        print('The key is: {}'.format(key))
        # Notice that the data in the dataframe is a string while the data in the key is a Timestamp
        # print('The key is: {}'.format(key))
        # Convert the data in the dataframe to a Timestamp
        df_k['DATA'] = pd.to_datetime(df_k['DATA'], format='%Y-%m-%d %H:%M:%S')
        return df_k[(df_k['SERIALE'] == key[0]) & (df_k['DATA'].dt.date == key[1])]

In [184]:
from geopy.geocoders import Nominatim
def get_coordinates_geopy(stop_name: str) -> list:
    """
        This function returns the coordinates of the stop given the stop_name.
        :param stop_name: the name of the stop
        :return: the coordinates of the stop
    """
    with open('stop_converted.json') as f:
        data_aux = json.load(f)
        # In data, sobstitute '-' with ' ' defining a new dictionary with the same keys and set it to lower case
        data = {k.lower().replace('-', ' '): v for k, v in data_aux.items()}
    # if stop_name contains a -, sobstitute it with a space; ignore aphostrophes and accents
    stop_name = stop_name.replace('-', ' ')
    stop_name = stop_name.replace('\'', '')
    stop_name = stop_name.replace('à', 'a')
    stop_name = stop_name.replace('è', 'e')
    stop_name = stop_name.replace('é', 'e')
    stop_name = stop_name.replace('ì', 'i')
    stop_name = stop_name.replace('ò', 'o')
    stop_name = stop_name.replace('ù', 'u')
    # If the last character of stop_name is a space, remove it
    if stop_name[-1] == ' ':
        stop_name = stop_name[:-1]

    if stop_name.lower() in data.keys() or stop_name.lower() + ' ' in data.keys():
        print('The stop {} is in the file stop_converted.json.'.format(stop_name.lower()))
        # The dictionary is in the format "stop_name": ["stop_id", "latitude", "longitude"]
        print('The coordinates of the stop {} are: {}'.format(stop_name.lower(), data[stop_name.lower()][1:]))
        return [float(data[stop_name.lower()][1]), float(data[stop_name.lower()][2])]
    elif 'san' in stop_name.lower():
        # if stop_name contains 'san', sobstitute it with 's' and try again
        stop_name = stop_name.lower().replace('san', 's.')
        if stop_name.lower() in data.keys() or stop_name.lower() + ' ' in data.keys():
            print('The stop {} is in the file stop_converted.json.'.format(stop_name.lower()))
            # The dictionary is in the format "stop_name": ["stop_id", "latitude", "longitude"]
            print('The coordinates of the stop {} are: {}'.format(stop_name.lower(), data[stop_name.lower()][1:]))
            return [float(data[stop_name.lower()][1]), float(data[stop_name.lower()][2])]
    elif len(stop_name.split(' ')) == 2:
            # if stop_name is composed by two words, try to find the first word
            print('The stop {} is composed by two words.'.format(stop_name) + 'Consider only the first word {}.'.format(stop_name.split(' ')[0]))
            stop_name_1 = stop_name.split(' ')[0]
            if stop_name_1.lower() in data.keys() or stop_name_1.lower() + ' ' in data.keys():
                print('The stop {} is in the file stop_converted.json.'.format(stop_name_1.lower()))
                # The dictionary is in the format "stop_name": ["stop_id", "latitude", "longitude"]
                print('The coordinates of the stop {} are: {}'.format(stop_name_1.lower(), data[stop_name_1.lower()][1:]))
                return [float(data[stop_name_1.lower()][1]), float(data[stop_name_1.lower()][2])]
            # if stop_name is composed by two words, try to find the second word
            print('The stop {} is composed by two words.'.format(stop_name) + 'Consider only the second word {}.'.format(stop_name.split(' ')[1]))
            stop_name_2 = stop_name.split(' ')[1]
            if stop_name_2.lower() in data.keys() or stop_name_2.lower() + ' ' in data.keys():
                print('The stop {} is in the file stop_converted.json.'.format(stop_name_2.lower()))
                # The dictionary is in the format "stop_name": ["stop_id", "latitude", "longitude"]
                print('The coordinates of the stop {} are: {}'.format(stop_name_2.lower(), data[stop_name_2.lower()][1:]))
                return [float(data[stop_name_2.lower()][1]), float(data[stop_name_2.lower()][2])]
    elif len(stop_name.split(' ')) == 3:
        # if stop_name is composed by three words, try to find the first two words
        print('The stop {} is composed by three words.'.format(stop_name) + 'Consider only the first two words {}.'.format(stop_name.split(' ')[0] + ' ' + stop_name.split(' ')[1]))
        stop_name = stop_name.split(' ')[0] + ' ' + stop_name.split(' ')[1]
        if stop_name.lower() in data.keys() or stop_name.lower() + ' ' in data.keys():
            print('The stop {} is in the file stop_converted.json.'.format(stop_name.lower()))
            # The dictionary is in the format "stop_name": ["stop_id", "latitude", "longitude"]
            print('The coordinates of the stop {} are: {}'.format(stop_name.lower(), data[stop_name.lower()][1:]))
            return [float(data[stop_name.lower()][1]), float(data[stop_name.lower()][2])]
    
    # If the stop is not in the file stop_all.json
    print('The stop {} is not in the dataframe.'.format(stop_name) + ' Try to find it with geopy.')

    geolocator = Nominatim(user_agent="my-app")
    location = geolocator.geocode(stop_name + ', Venezia')
    if location == None:
        print('The stop {} is not in geopy.'.format(stop_name))
        # Return the coordinates of the center of Venice
        print('The coordinates of the stop {} are: {}'.format(stop_name, [45.4371908, 12.3345898]))
        return [45.4371908, 12.3345898]
    # If coordinates are not in Veneto
    elif location.latitude < 44.0 or location.latitude > 46.0 or location.longitude < 11.0 or location.longitude > 13.0:
        print('The stop {} is not in Veneto.'.format(stop_name))
        # Return the coordinates of the center of Venice
        print('The coordinates of the stop {} are: {}'.format(stop_name, [45.4371908, 12.3345898]))
        return [45.4371908, 12.3345898]
    else:
        print('The coordinates of the stop {} are: {}'.format(stop_name, [location.latitude, location.longitude]))
        return [location.latitude, location.longitude]

In [185]:
# import json

# # Load the input JSON file
# with open('MASTER/transformData/stop_all.json', 'r') as file:
#     data = json.load(file)

# # Create a new dictionary for the transformed data
# transformed_data = {}

# # Iterate over the key-value pairs in the input data
# for stop_id, values in data.items():
#     stop_name = values[0]
#     latitude = values[1]
#     longitude = values[2]
#     transformed_data[stop_name] = [stop_id, latitude, longitude]

# # Write the transformed data to a new JSON file
# with open('output.json', 'w') as file:
#     json.dump(transformed_data, file)


In [186]:
from geopy.geocoders import Nominatim
def get_coordinates_geopy_vecchio(stop_name: str) -> list:
    """
        This function returns the coordinates of the stop given the stop_name.
        :param stop_name: the name of the stop
        :return: the coordinates of the stop
    """
    geolocator = Nominatim(user_agent="my-app")
    location = geolocator.geocode(stop_name + ', Venezia')
    if location == None:
        print('The stop {} is not in the dataframe.'.format(stop_name))
        return None
    else:
        # If location is not in the city of Venice
        # venice_latitude = 45.4371908
        # venice_longitude = 12.3345898
        if location.latitude < 45.4371908 or location.longitude < 12.3345898: # or location.longitude > 12.3345898:
            print('The coordinates of the stop {} are: {}'.format(stop_name, [location.latitude, location.longitude]))
            return [location.latitude, location.longitude]
        else:        
            # Return  TERRA" -1": ["TERRA", 45.491853, 12.242548]
            # print('The stop {} is not in the dataframe. It is set to the coordinates of the stop TERRA'.format(stop_name))
            # return [45.491853, 12.242548]

            # # Try to find the coordinates of the stop searching for the name of the stop in the file stop_all.json
            with open('MASTER/transformData/stop_all.json') as f:
                data = json.load(f)
            for stop in data:
                if 'stop_name' in stop and stop_name.lower() in stop['stop_name'].lower():
                    print('The coordinates of the stop {} are: {}'.format(stop_name, [stop['stop_lat'], stop['stop_lon']]))
                    return [stop['stop_lat'], stop['stop_lon']]

            # If the stop is not in the file stop_all.json
            print('The stop {} is not in the dataframe.'.format(stop_name))
            return [45.438667,12.319465]
                        

In [187]:
def get_coordinates (df_c: pd.DataFrame) -> pd.DataFrame:
    """
        This function returns the coordinates using openstreetmap.
        :param df_c: the dataframe
        :return: the coordinates
    """
    # There aren't columns with the coordinates so you have to obtain the coordinates using openstreetmap
    # Create a new column with the coordinates
    df_c['COORDINATES'] = df_c['DESCRIZIONE'].apply(get_coordinates_geopy)
    # Split the coordinates in two columns
    df_c[['LATITUDE', 'LONGITUDE']] = pd.DataFrame(df_c['COORDINATES'].tolist(), index=df_c.index)
    # Drop the column 'COORDINATES'
    df_c.drop(columns=['COORDINATES'], inplace=True)
    # If coordinate are "-1": ["TERRA", 45.491853, 12.242548]} change the description to "TERRA (fantom stop)" 
    # df.loc[df['LATITUDE'] == 45.491853, 'DESCRIZIONE'] = 'TERRA (fantom stop)'
    return df_c

In [188]:
# given a key, represent the trajectory of the user in the map
def plot_trajectory(df_c: pd.DataFrame, key: tuple, is_focus_on_ticket_code: bool = False, ticket_code: str = None, is_focus_on_tourists: bool = False, is_focus_on_workers: bool = False) -> None:
    """
        This function plots the trajectory of the user in the map.
        :param df: the dataframe
        :param key: the key
        :param is_focus_on_ticket_code: if True, the dictionary will be filtered by ticket code
        :param ticket_code: the ticket code to filter the dictionary
        :param is_focus_on_tourists: if True, the dictionary will be filtered by tourists
        :param is_focus_on_workers: if True, the dictionary will be filtered by workers
    """
    df_key = get_rows_from_key(df_c, key)
    # Get the coordinates of the trajectory
    df_key = get_coordinates(df_key)
    # Plot the trajectory
    fig, ax = plt.subplots(figsize=(15, 15))
    sns.scatterplot(x='LONGITUDE', y='LATITUDE', data=df_key, hue='DESCRIZIONE', s=100, ax=ax)
    # Set the title
    if is_focus_on_ticket_code:
        ax.set_title('Trajectory of the user {} with ticket code {}'.format(key[0], ticket_code))
    elif is_focus_on_tourists:
        ax.set_title('Trajectory of the tourist {}'.format(key[0]))
    elif is_focus_on_workers:
        ax.set_title('Trajectory of the worker {}'.format(key[0]))
    else:
        ax.set_title('Trajectory of the user {}'.format(key[0]))
    # Set the legend
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1)
    # Set the x and y labels
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    # Set the x and y ticks
    ax.set_xticks(np.arange(12.2, 12.5, 0.01))
    ax.set_yticks(np.arange(45.4, 45.6, 0.01))
    # Set the grid
    ax.grid(True)
    # Set the aspect ratio
    ax.set_aspect('equal', 'box')   

In [189]:
# Function: given key, obtain the trajectory of the user and represent it in the map using folium
# GIUSTA
# Inizialize map and list empty
def represent_trajectory_on_map (df_c: pd.DataFrame, key: tuple) -> None:
    """
        This function represents the trajectory of the user in the map.
        :param df: the dataframe
        :param key: the key
        :param map: the map
        :param stop_inserted: the list of stops already inserted
        :param marker_cluster: the marker cluster
        :return: the map
    """

    df_key = get_rows_from_key(df_c, key)
    # Get the coordinates of the trajectory using openstreetmap
    df_key = get_coordinates(df_key)


    # print(df_key.head(50))
    
    # Create a map
    m = folium.Map(location=[45.437190, 12.334590], zoom_start=13)
    
    marker_cluster = MarkerCluster().add_to(m)
    
    # Add markers to the map only if there isn't already a marker in the same position
    # First stop is green, last stop is red, other stops are blue
    # Insert first to all the first and last stops
    # If the first stop is the same of the last stop, insert only the first stop with black color

    # Define a list of stops already inserted
    stops_inserted = []
   
    if df_key.iloc[0]['LATITUDE'] == df_key.iloc[-1]['LATITUDE'] and df_key.iloc[0]['LONGITUDE'] == df_key.iloc[-1]['LONGITUDE']:
        print('The first stop is the same of the last stop.')
        folium.Marker(location=[df_key.iloc[0]['LATITUDE'], df_key.iloc[0]['LONGITUDE']], popup=df_key.iloc[0]['DESCRIZIONE'], icon=folium.Icon(color='black')).add_to(marker_cluster)
    else:
        folium.Marker(location=[df_key.iloc[0]['LATITUDE'], df_key.iloc[0]['LONGITUDE']], popup=df_key.iloc[0]['DESCRIZIONE'], icon=folium.Icon(color='green')).add_to(marker_cluster)
        # If the first stop is the same of the last stop, insert only the first stop
        folium.Marker(location=[df_key.iloc[-1]['LATITUDE'], df_key.iloc[-1]['LONGITUDE']], popup=df_key.iloc[-1]['DESCRIZIONE'], icon=folium.Icon(color='red')).add_to(marker_cluster)
    
    # Add the first and the last stop to the list of stops already inserted
    stops_inserted.append((df_key.iloc[0]['LATITUDE'], df_key.iloc[0]['LONGITUDE']))
    # If the first stop is the same of the last stop, insert only the first stop
    if df_key.iloc[0]['LATITUDE'] != df_key.iloc[-1]['LATITUDE'] and df_key.iloc[0]['LONGITUDE'] != df_key.iloc[-1]['LONGITUDE']:
        stops_inserted.append((df_key.iloc[-1]['LATITUDE'], df_key.iloc[-1]['LONGITUDE']))
        

    # Insert the other stops
    for i in range(1, len(df_key)-1):
        # If the stop is not already inserted, insert it
        if (df_key.iloc[i]['LATITUDE'], df_key.iloc[i]['LONGITUDE']) not in stops_inserted:
            stops_inserted.append((df_key.iloc[i]['LATITUDE'], df_key.iloc[i]['LONGITUDE']))
            folium.Marker(location=[df_key.iloc[i]['LATITUDE'], df_key.iloc[i]['LONGITUDE']], popup=df_key.iloc[i]['DESCRIZIONE']).add_to(marker_cluster)        

    # Highlight the trajectory of the user
    folium.PolyLine(locations=df_key[['LATITUDE', 'LONGITUDE']].values.tolist(), weight=2, color='red').add_to(m)

    # Insert the legend with the title Legend in bold
    legend_html = '''
        <div style="position: fixed;
                    bottom: 50px; left: 50px; width: 300px; height: 120px;
                    border:2px solid grey; z-index:9999; font-size:14px;
                    ">&nbsp; <b><u>Legend</u></b> <br>
                        &nbsp; First stop &nbsp; <i class="fa fa-map-marker fa-2x" style="color:green"></i><br>
                        &nbsp; Last stop &nbsp; <i class="fa fa-map-marker fa-2x" style="color:red"></i><br>
                        &nbsp; Coincident first and last stop &nbsp; <i class="fa fa-map-marker fa-2x" style="color:black"></i><br> 
        </div>
        '''
    m.get_root().html.add_child(folium.Element(legend_html))
    

    # Display the map
    display(m)

    # Save the map (if the folder doesn't exist, create it)
    # import os
    # if not os.path.exists('data/maps/' + subfolder):
    #     os.makedirs('data/maps/' + subfolder)
    # m.save('data/maps/' + subfolder + '/map_' + file_name + '_key:' + str(key) + '.html')

In [190]:
# Open the dictionary of trajectories
dict_trajectories = open_dict_trajectories(is_focus_on_ticket_code=True, ticket_code='1')

In [191]:
# Print the head and the tail of the dictionary
for key in list(dict_trajectories.keys())[:5] + list(dict_trajectories.keys())[-5:]:
    print(key, dict_trajectories[key])

(40834225654428164, None) [['SAN MARCO', 'RIALTO', 'P.LE ROMA', 'ZITELLE', 'ZITELLE', 'ZATTERE', 'ZITELLE']]
(40553391836814084, None) [['P.LE ROMA', 'F.TE NOVE', 'BURANO', 'RIALTO', 'RIALTO']]
(40553391836814340, None) [['P.LE ROMA', 'F.TE NOVE', 'BURANO', 'RIALTO', 'RIALTO']]
(41679291734809348, None) [['P.LE ROMA', 'P.LE ROMA']]
(41679291734809604, None) [['P.LE ROMA']]
(40553621743968516, None) [['TRONCHETTO']]
(40553391555332612, None) [['P.LE ROMA']]
(40553391555332868, None) [['P.LE ROMA']]
(40835096720245252, None) [['P.LE ROMA']]
(40553621743596292, None) [['P.LE ROMA']]


In [192]:
first_key = list(dict_trajectories.keys())[0]
first_key = extract_values_key(first_key)

# Get the rows of the dataframe for the first key
df_first_key = get_rows_from_key(df, first_key)
df_first_key.head(50)

The key is: (40834225654428164, None)


,DATA,ORA,DATA_VALIDAZIONE,SERIALE,FERMATA,DESCRIZIONE,TITOLO,TICKET_CODE,DESCRIZIONE_TITOLO
30,2023-01-13,00:06:00,2023-01-13 00:06:00,40834225654428164,5011,SAN MARCO,11261,1,"DAILYP-TPL19,90-C.VE5,10"
52994,2023-01-13,17:24:00,2023-01-13 17:24:00,40834225654428164,5039,RIALTO,11261,1,"DAILYP-TPL19,90-C.VE5,10"
55459,2023-01-13,17:55:00,2023-01-13 17:55:00,40834225654428164,5028,P.LE ROMA,11261,1,"DAILYP-TPL19,90-C.VE5,10"
61767,2023-01-13,19:36:00,2023-01-13 19:36:00,40834225654428164,5081,ZITELLE,11261,1,"DAILYP-TPL19,90-C.VE5,10"
76238,2023-01-14,09:38:00,2023-01-14 09:38:00,40834225654428164,5081,ZITELLE,11261,1,"DAILYP-TPL19,90-C.VE5,10"
92756,2023-01-14,12:38:00,2023-01-14 12:38:00,40834225654428164,5049,ZATTERE,11261,1,"DAILYP-TPL19,90-C.VE5,10"
101267,2023-01-14,14:14:00,2023-01-14 14:14:00,40834225654428164,5081,ZITELLE,11261,1,"DAILYP-TPL19,90-C.VE5,10"


In [193]:
# Represent the trajectory of the user in the map
represent_trajectory_on_map(df, first_key)

The key is: (40834225654428164, None)
The stop s. marco is in the file stop_converted.json.
The coordinates of the stop s. marco are: ['45.432125', '12.337271']
The stop rialto is in the file stop_converted.json.
The coordinates of the stop rialto are: ['45.437584', '12.335558']
The stop p.le roma is in the file stop_converted.json.
The coordinates of the stop p.le roma are: ['45.438667', '12.319465']
The stop zitelle is in the file stop_converted.json.
The coordinates of the stop zitelle are: ['45.426720', '12.337905']
The stop zitelle is in the file stop_converted.json.
The coordinates of the stop zitelle are: ['45.426720', '12.337905']
The stop zattere is in the file stop_converted.json.
The coordinates of the stop zattere are: ['45.429260', '12.326280']
The stop zitelle is in the file stop_converted.json.
The coordinates of the stop zitelle are: ['45.426720', '12.337905']


In [194]:
# Given a dataframe, with the coordinates of the stops and the trajectories of different users, represent the trajectories of the users in the map using folium
# Each user has a different color and the first and the last stop are highlighted
# list_keys is a list of keys of the dictionary of trajectories (list of tuples)
def represent_trajectories_on_map_users (df_c: pd.DataFrame, list_keys:list) -> None:
    """
        This function represents the trajectories of the users in the map.
        :param df: the dataframe
        :param list_keys: the list of keys of the dictionary of trajectories
    """

    # Get a list of colors
    colors = ['red', 'blue', 'green', 'black', 'orange', 'purple', 'darkred', 'lightred', 'beige', 'darkblue', 
              'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 
              'lightgray', 'blue', 'green', 'red', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 
              'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']

    # Create a map
    m = folium.Map(location=[45.437190, 12.334590], zoom_start=13)

    # Create a marker cluster
    marker_cluster = MarkerCluster().add_to(m)

    # Define a list of stops already inserted
    # Each element of the list is a tuple (latitude, longitude)
    stops_inserted = []
    
    j = 0

    # For each key in the list of keys
    for key in list_keys:
        if j == len(colors):
            j = 0
        # Extract the values of the key
        key = extract_values_key(key)
        df_key = get_rows_from_key(df_c, key)
        # Get the coordinates of the trajectory
        df_key = get_coordinates(df_key)

        # Add markers to the map only if there isn't already a marker in the same position
        for i in range(0, len(df_key)):
            # If the stop is not already inserted, insert it
            if (df_key.iloc[i]['LATITUDE'], df_key.iloc[i]['LONGITUDE']) not in stops_inserted:
                stops_inserted.append((df_key.iloc[i]['LATITUDE'], df_key.iloc[i]['LONGITUDE']))
                folium.Marker(location=[df_key.iloc[i]['LATITUDE'], df_key.iloc[i]['LONGITUDE']], popup=df_key.iloc[i]['DESCRIZIONE']).add_to(marker_cluster)

        # Highlight the trajectory of the user
        folium.PolyLine(locations=df_key[['LATITUDE', 'LONGITUDE']].values.tolist(), weight=2, color=colors[j]).add_to(m)


        j += 1

    # Display the map
    display(m)

    # Save the map (if the folder doesn't exist, create it)
    import os
    if not os.path.exists('data/maps/' + subfolder):
        os.makedirs('data/maps/' + subfolder)
    m.save('data/maps/' + subfolder + '/map_' + file_name + '_users.html')


        

represent_trajectories_on_map_users(df, list(dict_trajectories.keys())[150:300])




The key is: (40553391835911428, None)
The stop zattere is in the file stop_converted.json.
The coordinates of the stop zattere are: ['45.429260', '12.326280']
The stop f.te nove is in the file stop_converted.json.
The coordinates of the stop f.te nove are: ['45.443016', '12.340888']
The stop burano is in the file stop_converted.json.
The coordinates of the stop burano are: ['45.487602', '12.416217']
The stop s. marco is in the file stop_converted.json.
The coordinates of the stop s. marco are: ['45.432125', '12.337271']
The key is: (40553391835911684, None)
The stop zattere is in the file stop_converted.json.
The coordinates of the stop zattere are: ['45.429260', '12.326280']
The stop f.te nove is in the file stop_converted.json.
The coordinates of the stop f.te nove are: ['45.443016', '12.340888']
The stop burano is in the file stop_converted.json.
The coordinates of the stop burano are: ['45.487602', '12.416217']
The stop s. marco is in the file stop_converted.json.
The coordinates o

In [199]:
# select a user that has a trajectory with the stop "people mover" 

user = df[df['DESCRIZIONE'] == 'PEOPLE MOVER']['SERIALE'].unique()[6]
print(user)

# represent the trajectory of the user in the map
represent_trajectory_on_map(df, (user, None))


-3613121932
The key is: (-3613121932, None)
The stop people mover is in the file stop_converted.json.
The coordinates of the stop people mover are: [45.4371908, 12.3345898]
The stop people mover is in the file stop_converted.json.
The coordinates of the stop people mover are: [45.4371908, 12.3345898]
The stop p.le roma is in the file stop_converted.json.
The coordinates of the stop p.le roma are: ['45.438667', '12.319465']
The stop people mover is in the file stop_converted.json.
The coordinates of the stop people mover are: [45.4371908, 12.3345898]
The stop people mover is in the file stop_converted.json.
The coordinates of the stop people mover are: [45.4371908, 12.3345898]
The stop p.le roma is in the file stop_converted.json.
The coordinates of the stop p.le roma are: ['45.438667', '12.319465']
The stop people mover is in the file stop_converted.json.
The coordinates of the stop people mover are: [45.4371908, 12.3345898]
The stop people mover is in the file stop_converted.json.
The